# 問題
25の処理時に、テンプレートの値からMediaWikiの強調マークアップ（弱い強調、強調、強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）。

In [5]:
import pandas as pd
import re

df = pd.read_json("Jawiki Country.json.gz", lines=True)
uk_text = df.query('title == "イギリス"')["text"].values[0]

# 1) 基礎情報ブロックを抜く（終端の空白も許容）
block_pat = r'^\{\{\s*基礎情報[^\n]*.*?^\}\}\s*$'
blocks = re.findall(block_pat, uk_text, flags=re.DOTALL | re.MULTILINE)
if not blocks:
    raise ValueError("基礎情報ブロックが見つかりません")
block = blocks[0]

# 2) フィールド抽出：次のフィールド行 or 終端 or 文字列末まで
field_pat = r'^\|\s*(.+?)\s*=\s*(.*?)(?=\n\||\n\}\}|$)'

# 3) 強調マークアップ除去関数（抽出後に適用）
def remove_emphasis(text: str) -> str:
    patterns = [
        (r"'''''(.*?)'''''", r"\1"),  # 強い強調(pat, repl)
        (r"'''(.*?)'''",     r"\1"),  # 強調
        (r"''(.*?)''",       r"\1"),  # 弱い強調
    ]
    changed = True
    while changed:
        old = text
        for pat, repl in patterns:
            text = re.sub(pat, repl, text, flags=re.DOTALL)
        changed = (text != old)
    return text.strip()

# 4) 辞書化（組み込み名を上書きしない）
info = {}
for m in re.finditer(field_pat, block, flags=re.DOTALL | re.MULTILINE):
    name = m.group(1).strip()
    value = m.group(2).strip()
    value = remove_emphasis(value)  # ← ここで強調を除去
    info[name] = value

info

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（[[イギリスの国章|国章]]）',
 '標語': '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）',
 '国歌': '[[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br />神よ女王を護り賜え<br />{{center|[[ファイル:United States Navy Band - God Save the Queen.ogg]]}}',
 '地図画像': 'Europe-UK.svg',
 '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg',
 '公用語': '[[英語]]',
 '首都': '[[ロンドン]]（事実上）',
 '最大都市': 'ロンドン',
 '元首等肩書': '[[イギリスの君主|女王]]',
 '元首等氏名': '[[エリザベス2世]]',
 '首相等肩書': '[[イギリスの首相|首相]]',
 '首相等氏名': '[[ボリス・ジョンソン]]',
 '他元首等肩書1': '[[貴族院 (イギリス)|貴族院議長]]',
 '他元首等氏名1': '[[:en:Norman Fowler, Baron Fowler|ノーマン・ファウラー]]',
 '他元首等肩書2': '[[庶民院 (イギリス)|庶民院議長]]',
 '他元首等氏名2': '{{仮リンク|リンゼイ・ホイル|en|Lindsay H